In [2]:
from visualization.utils import display_img
from skimage import io, exposure
import numpy as np
import cv2 as cv
import matplotlib.pyplot as plt
import time

## Visualization

In [22]:
stpt_img = io.imread('data/train/STPT/S014_Z00.tif')
# plt.imshow(img, cmap='gray')

imc_img = io.imread('data/train/IMC/SECTION_01/131Xe.tif')

Shape of STPT image: (18720, 18720)
Shape of IMC image: (18720, 18720)


In [31]:
imc_img = torch.from_numpy(imc_img.astype(np.int16))
imc_img = torch.unsqueeze(imc_img, 0)

stpt_img = torch.from_numpy(stpt_img).permute((2,0,1))

torch.Size([4, 20800, 20800])

In [40]:
stpt_img = torchvision.transforms.Resize(18720)(stpt_img)
combine = torch.cat((stpt_img, imc_img), 0)
combine.shape

torch.Size([5, 18720, 18720])

In [73]:
x = [torch.split(combine, 4)]
y = map(list, zip(*x))

In [80]:
type(x[0])

tuple

## Concatenating IMC images

In [12]:
import os, shutil
import torch
import cv2 as cv
import numpy as np
import time
from multiprocessing.pool import Pool
from skimage import io

In [13]:
def process_imc_image(file_name, plot=False, bits=8, v=[0, 256]):
    # read image file
    img = cv.imread(file_name, cv.IMREAD_UNCHANGED)

    # normalize image
    norm_img = img.copy()
    cv.normalize(img, norm_img, alpha=0, beta=2**bits - 1, norm_type=cv.NORM_MINMAX)

    # Apply log transformation method
    c = (2**bits - 1) / np.log(1 + np.max(norm_img))
    log_image = c * (np.log(norm_img + 1))
    
    if plot:
        plt.imshow(log_image, cmap='gray', vmin=v[0], vmax=v[1])
    else:
        # Specify the data type so that
        # float value will be converted to int
        return torch.from_numpy(log_image)

    
imc_section_01_folder = 'data/train/IMC/SECTION_01/'
imc_imgs = [os.path.join(imc_section_01_folder, imc_img)
            for imc_img in os.listdir(imc_section_01_folder)
            if imc_img.endswith('.tif')]

In [14]:
start = time.time()
with Pool(maxtasksperchild=100) as p:
    imc_imgs = list(p.imap(process_imc_image, imc_imgs))

end = time.time()
print('took', end-start, 'seconds')

took 11.589147090911865 seconds


In [ ]:
imc_imgs = [torch.unsqueeze(img, 0) for img in imc_imgs]
imc_imgs = torch.cat(imc_imgs, 0)

In [ ]:
def process_stpt_image(file_name):
    img = io.imread(file_name)
    return torch.from_numpy(img)
    
stpt_img_paths = [os.path.join('data/train/STPT/',
                               'S{0}_Z{1}.tif'.format(str(1).zfill(3),
                                                  optical_section.zfill(2)))
                  for optical_section in ['0', '1']]

start = time.time()
with Pool(maxtasksperchild=100) as p:
    stpt_imgs = list(p.imap(process_stpt_image, stpt_img_paths))
end = time.time()
print('Loading STPT images took', end-start, 'seconds')

In [1]:
stpt_imgs = [img.permute((2,0,1)) for img in stpt_imgs]
print(stpt_imgs[0].shape)
print(imgs[0].shape)

NameError: name 'stpt_imgs' is not defined